# Adição do Campo de Classe Processual no Modelo Truncated

Nesse notebook será avaliada a influência da classe processual do acórdão, representado pelo campo 'tipo_acordao', na eficácia do modelo FrontBack para classificação de acórdãos dentre os ramos do direito.

Para isso, será feito um treinamento de modelo com aproximação FrontBack juntando os textos das ementas junto com os textos das classes processuais. Por fim é feita uma pequena avaliação do modelo baseada em métricas básicas.

## Inicialização e definiçao de constantes

Como uma etapa inicial, toda a inicialização do notebook será concentrada no início desse documento. Os conteúdos contidos aqui são:

1. Instalação de bibliotecas externas
2. Importação de biblioteca
3. Definição de valores constantes que podem ter seu uso replicado ao longo do notebook
4. Inicialização do sistema de arquivos integrado ao Google Drive

In [ ]:
# Installation of 3rd party libraries

!pip install transformers
!pip install --upgrade pytorch-lightning

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 59 kB 5.4 MB/s 
     |████████████████████████████████| 895 kB 18.0 MB/s 
     |████████████████████████████████| 3.3 MB 41.0 MB/s 
     |████████████████████████████████| 596 kB 33.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 329 kB 41.1 MB/s 
     |████████████████████████████████| 132 kB 48.1 MB/s 
     |████████████████████████████████| 829 kB 32.8 MB/s 
     |████████████████████████████████| 1.1 MB 36.3 MB/s 
     |████████████████████████████████| 160 kB 44.5 MB/s 
     |████████████████████████████████| 271 kB 47.8 MB/s 
     |████████████████████████████████| 192 kB 47.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=163b179cc9b94b

In [ ]:
# Imports

from google.colab import drive
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast as BertTokenizer, BertModel, BertForSequenceClassification, AdamW, get_cosine_schedule_with_warmup, Trainer, TrainingArguments
from enum import Enum
from typing import List
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler
import re
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, cohen_kappa_score, matthews_corrcoef, balanced_accuracy_score

In [ ]:
# Constants

CONSTANTS = {
    'TRAINING_DATASET': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Train_Dataset.csv',
    'VALIDATION_DATASET': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Validation_Dataset.csv',
    'TEST_DATASET': '/content/drive/My Drive/MAC499 - Kaique e Yurick/DB/Test_Dataset.csv',
    'BERT_MODEL_NAME': 'neuralmind/bert-large-portuguese-cased',
    'SEED': 13,
    'MODEL_PATH': '/content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/saved_models/frontback_finetuned.bin'
}

# Hyperparameters

HYPERPARAMETERS = {
    'BATCH_SIZE': 2,
    'EPOCHS': 5,
    'MAX_NUMBER_TOKENS': 512,
    'LEARNING_RATE': 2e-5,
    'NUMBER_OF_BRANCHES': 13,
    'WEIGHT_DECAY': 0.01,
    'WARM_UP_PROPORTION': 0.3,
    'INTRODUCTION_WORDS': 256
}

In [ ]:
# Mounting Google Drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Verificar disponibilidade da GPU

O próximo passo seria verificar se a GPU oferida pela Google gratuitamente como ambiente de execução do notebook está funcionando corretamente. A GPU oferece uma performance computacional maior em relação a calculos sendo executados pela CPU.

In [ ]:
torch.cuda.empty_cache()

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


### Reproducibilidade

Para fins de reproducibilidade, podemos definir uma semente para o pytorch lightning.

In [ ]:
seed_everything(CONSTANTS['SEED'])

Global seed set to 13


13

## Carregar os dados

Após as configurações iniciais, os dados são carregados da mesma forma que no notebook de criação e treinamento do modelo FrontBack.

Assim, nessa etapa são carregados arquivos .csv contendo os conjuntos de treinamento e de validação.

In [ ]:
# Read the training dataset from .csv file
documents = pd.read_csv(CONSTANTS['TRAINING_DATASET'])
# Read the validation dataset from .csv file
documents_val = pd.read_csv(CONSTANTS['VALIDATION_DATASET'])

In [ ]:
def get_words(sentence: str) -> List[str]:
    """Retrieves the words in a sentence by splitting into blank spaces.

    Args:
        sentence (str): a sentence.

    Returns:
        List[str]: list of words in sentence.
    """
    return re.findall(r'\S+', sentence)

def merge_front_back_summary(full_summary: str, max_summary_size: int) -> str:
    """Extracts the part of the front and part of the back from the summary,
    following the properties defined in Constants structure.

    Args:
        full_summary (str): the entire summary content.
        max_summary_size (int): the max number of tokens considered.

    Returns:
        str: a version of summary with the beginning of its front and the end
        of its back.
    """
    front_size = HYPERPARAMETERS['INTRODUCTION_WORDS']
    back_size = max_summary_size - front_size

    summary_words = get_words(full_summary)

    if (len(summary_words) <= max_summary_size):
        return full_summary

    front_summary = summary_words[:front_size]
    back_summary = summary_words[(len(summary_words) - back_size):]

    return ' '.join(front_summary).strip() + ' ' + ' '.join(back_summary).strip()

In [ ]:
# Training dataset: Change column containing summary to considers front and back of it
documents['ementa'] = documents['ementa'].apply(lambda summary: merge_front_back_summary(summary, HYPERPARAMETERS['MAX_NUMBER_TOKENS']))
documents = documents.iloc[: , 1:]

In [ ]:
# Validation dataset: Change column containing summary to considers front and back of it
documents_val['ementa'] = documents_val['ementa'].apply(lambda summary: merge_front_back_summary(summary, HYPERPARAMETERS['MAX_NUMBER_TOKENS']))
documents_val = documents_val.iloc[: , 1:]

O objetivo desse notebook é a utilização dos dados presentes no campo 'tipo_acordao' dos acórdãos, o que não foi feito no notebook de criação do modelo FrontBack.

A informação se encontra em formato de sigla no conjunto de dados, o que não ajudaria o modelo a fazer classificações a partir da ementa. Portanto, é necessário efetuar um pré-processamento dos dados, mapeando as siglas das classes processuais para os seus respectivos significados, contidos no Enum criado. Após isso, é criado um novo dataframe contendo apenas os valores das classes processuais processadas e seus respectivos valores de classificação entre os ramos.

In [ ]:
# Definition of mapping from law branch name to a numeric identifier

class ProcessualClass(Enum):
    """Mapping to a Processual Class and an identification."""

    HC = 'Habeas Corpus'
    RE = 'Recurso Extraordinário'
    MS = 'Mandado de Segurança'
    ADI = 'Ação Direta de Inconstitucionalidade'
    RHC = 'Recurso Ordinário em Habeas Corpus'
    EXT = 'Extradição'
    RMS = 'Recurso Ordinário em Mandado de Segurança'
    ADPF = 'Arguição de Descumprimento de Preceito Fundamental'
    INQ = 'Inquérito'
    AR = 'Ação Rescisória'
    ARE = 'Recurso Extraordinário com Agravo'
    PET = 'Petição'
    AP = 'Ação Penal'
    MI = 'Mandado de Injunção'
    RVC = 'Revisão Criminal'
    AI = 'Agravo de Instrumento'
    AO = 'Ação Originária'
    IF = 'Intervenção Federal'
    CC = 'Conflito de Competência'
    ADC = 'Ação Declaratória de Constitucionalidade'
    MC_ADI = 'Medida Cautelar em Ação Direta de Inconstitucionalidade'
    ACO = 'Ação Cível Originária'
    ED_RE = 'Embargos de Declaração em Recursos Extraordinários'
    ADO = 'Ação Direta de Insconstitucionalidade por Omissão'
    AC = 'Ação Cautelar'
    AGR_RE = 'Agravo Regimental em Recurso Extraordinário'
    ED_ADI = 'Embargos de Declaração na Ação Direta de Inconstitucionalidade'
    QO_RG_AGR_RE = 'Questão de Ordem na Repercussão Geral de Agravo Regimental em Recurso Extraordinário'
    PSV = 'Proposta de Súmula Vinculante'
    RHD = 'Recurso de Habeas-Data'
    ED_QO_ED_RMS = 'Embargos de Declaração na Questão de Ordem nos Embargos de Declaração no Recurso Ordinário em Mandado de Segurança'
    TPA = 'Tutela Provisória Antecedente'
    AOE = 'Ação Originária Especial'
    ED_RHC = 'Embargos de Declaração em Recurso Ordinário em Habeas Corpus'
    ED_ED_EXT = 'Embargos de Declaração nos Embargos de Declaração na Extradição'
    ED_INQ = 'Embargos de Declaração no Inquérito'
    AGR_MS = 'Agravo Regimental em Mandado de Segurança'
    RCL = 'Reclamação'
    ED_AGR_AI = 'Embargos de Declaração em Agravo Regimental em Agravo de Instrumento'


    def get_identifier(self) -> str:
        """Retrieves the identifier letters for this instance of Processual Class.

        Returns:
            str: identifier of this instance of ProcessualClass
        """
        return self.value
    
    @staticmethod
    def get_value_from_name(name) -> str:
      """"Retrieves a processual class from a name"""
      for proc_class in ProcessualClass:
        if proc_class.name == name:
          return proc_class.value

In [ ]:
ramos = []
classes = []
for i, proc_class in enumerate(documents['tipo_acordao']):
  if proc_class.strip() != '':
    value = ProcessualClass.get_value_from_name(proc_class.replace(' ', '_'))
    classes.append(value)
    ramos.append(documents['ramo'][i])
documents_train = pd.DataFrame(list(zip(ramos, classes)),
               columns = ['ramo', 'text'])
for row in documents.iterrows():
  tmp = pd.DataFrame({'ramo': [row[1].ramo], 'text': [row[1].ementa]})
  documents_train = pd.concat([documents_train, tmp], ignore_index = True, axis = 0)
del documents
documents_train

,ramo,text
0,1,Mandado de Segurança
1,1,Ação Direta de Inconstitucionalidade
2,1,Ação Direta de Inconstitucionalidade
3,0,Habeas Corpus
4,10,Recurso Ordinário em Habeas Corpus
...,...,...
7727,2,EMENTA Recurso extraordinário – Emenda Constit...
7728,1,EMENTA: MANDADO DE SEGURANÇA. DECRETO HOMOLOGA...
7729,0,EMENTA: RECURSO ORDINÁRIO EM HABEAS-CORPUS. PR...
7730,1,Ementa: DIREITO CONSTITUCIONAL E ADMINISTRATIV...


In [ ]:
ramos_val = []
classes_val = []
for i, proc_class in enumerate(documents_val['tipo_acordao']):
  if proc_class != '':
    value = ProcessualClass.get_value_from_name(proc_class.replace(' ', '_'))
    classes_val.append(value)
    ramos_val.append(documents_val['ramo'][i])
documents_validation = pd.DataFrame(list(zip(ramos_val, classes_val)),
               columns = ['ramo', 'text'])
for row in documents_val.iterrows():
  tmp = pd.DataFrame({'ramo': [row[1].ramo], 'text': [row[1].ementa]})
  documents_validation = pd.concat([documents_validation, tmp], ignore_index = True, axis = 0)
del documents_val
documents_validation

,ramo,text
0,4,Recurso Extraordinário
1,0,Habeas Corpus
2,9,Extradição
3,0,Habeas Corpus
4,0,Recurso Ordinário em Habeas Corpus
...,...,...
1653,0,EMENTA: PENAL E PROCESSUAL PENAL. RECURSO ORDI...
1654,1,Ementa: CONSTITUCIONAL E ADMINISTRATIVO. MANDA...
1655,1,Ementa: DIREITO CONSTITUCIONAL. ADI. VINCULAÇÃ...
1656,0,Ementa: PETIÇÃO. AGRAVO REGIMENTAL. DENÚNCIA. ...


O treinamento do modelo nesse notebook segue o mesmo formato do notebook de criação do modelo FrontBack. Assim, é feito o download do tokenizer e são definidas e criadas classes para Dataset, DataModule e pro modelo em si.

Por fim, o é feito o treinamento utilizando os novos dataframes contendo as indexações misturadas com as ementas como input.

In [ ]:
class LawDocumentDataset(Dataset):
  def __init__(self, dataframe: pd.DataFrame, tokenizer: BertTokenizer, max_token_length: int=512):
    self.dataframe = dataframe
    self.tokenizer = tokenizer
    self.max_token_length = max_token_length

  def __len__(self):
    return len(self.dataframe)

  def __getitem__(self, index: int):
    row = self.dataframe.iloc[index]
    summary_document = row.text
    law_branch_id = row.ramo

    encoding = self.tokenizer.encode_plus(
      summary_document,
      add_special_tokens=True,          # Add `[CLS]` and `[SEP]`
      max_length=self.max_token_length,
      return_token_type_ids=False,
      padding="max_length",
      truncation=True,                  # Truncate encoding to the max length
      return_attention_mask=True,       # Return attention mask
      return_tensors="pt"               # Return PyTorch tensor
    )

    labels = np.eye(HYPERPARAMETERS['NUMBER_OF_BRANCHES'])[law_branch_id]  # Return a list with zeros, except for index law_branch_id that assumes one

    return dict(
        summary_document=summary_document,
        input_ids=encoding["input_ids"].flatten(),
        attention_mask=encoding["attention_mask"].flatten(),
        labels=torch.FloatTensor(labels)
    )

In [ ]:
tokenizer = BertTokenizer.from_pretrained(CONSTANTS['BERT_MODEL_NAME'])

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [ ]:
train_dataset = LawDocumentDataset(documents_train, tokenizer, HYPERPARAMETERS['MAX_NUMBER_TOKENS'])
validation_dataset = LawDocumentDataset(documents_validation, tokenizer, HYPERPARAMETERS['MAX_NUMBER_TOKENS'])

In [ ]:
class LawDocumentClassifier(pl.LightningModule):
    
    def __init__(self, number_classes: int, steps_per_epoch: int=None, epochs: int=None, learning_rate: float=2e-5, weight_decay: float=0.01, warm_up_proportion: float=0.3):
        super().__init__()
        
        self.model = BertForSequenceClassification.from_pretrained(
            "neuralmind/bert-large-portuguese-cased",
            num_labels=number_classes,                      # The number of output labels--2 for binary classification
            output_attentions=False,                        # Returns attention weights
            output_hidden_states=False                      # Returns all hidden states
        )
        self.steps_per_epoch = steps_per_epoch
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.warm_up_proportion = warm_up_proportion
        self.weight_decay = weight_decay
        
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids,
                            attention_mask=attention_mask,
                            labels=labels,
                            return_dict=True)
        
        return output.loss, output.logits
        
    def training_step(self, batch, batch_index):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        
        loss, outputs = self(input_ids, attention_mask, labels)
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": outputs, "labels": labels}

    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        
        classification_predictions = self.convert_to_classification_labels(logits)
        classification_labels = self.convert_to_classification_labels(labels)

        metrics = {
            "validation_accuracy": accuracy_score(classification_labels, classification_predictions),
            "validation_balanced_accuracy": balanced_accuracy_score(classification_labels, classification_predictions),
            "validation_precision": precision_score(classification_labels, classification_predictions, average='weighted'),
            "validation_recall": recall_score(classification_labels, classification_predictions, average='weighted'),
            "validation_f1": f1_score(classification_labels, classification_predictions, average='weighted'),
            "validation_cohen_kappa": cohen_kappa_score(classification_labels, classification_predictions),
            "validation_matthews": matthews_corrcoef(classification_labels, classification_predictions)
        }

        return metrics
            
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        warmup_steps = int(self.steps_per_epoch * self.warm_up_proportion)
        total_steps = self.steps_per_epoch * self.epochs - warmup_steps

        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        
        return (optimizer, scheduler)

    def convert_to_classification_labels(self, classifications):
        formatted_classifications = []

        for classification in classifications:
            formatted_classifications.append(np.argmax(classification).flatten())

        return formatted_classifications

In [ ]:
model = LawDocumentClassifier(
    HYPERPARAMETERS['NUMBER_OF_BRANCHES'],
    steps_per_epoch=len(documents_train) // HYPERPARAMETERS['BATCH_SIZE'],
    epochs=HYPERPARAMETERS['EPOCHS'],
    learning_rate=HYPERPARAMETERS['LEARNING_RATE'],
    weight_decay=HYPERPARAMETERS['WEIGHT_DECAY'],
    warm_up_proportion=HYPERPARAMETERS['WARM_UP_PROPORTION']
)

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from th

In [ ]:
training_args = TrainingArguments(
    "/content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/processual_class_trainer_output",
    num_train_epochs = HYPERPARAMETERS['EPOCHS'],
    evaluation_strategy = 'epoch',
    per_device_train_batch_size = HYPERPARAMETERS['BATCH_SIZE'],
    logging_steps = 30,
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=model.compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=model.configure_optimizers()
)
trainer.train()

***** Running training *****
  Num examples = 7732
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 11598


Epoch,Training Loss,Validation Loss,Validation Accuracy,Validation Precision,Validation Recall,Validation F1
1,0.144300,0.109818,0.747889,0.736459,0.747889,0.709173
2,0.084600,0.102639,0.762364,0.764882,0.762364,0.733400
3,0.062700,0.103317,0.759952,0.768837,0.759952,0.735094


Saving model checkpoint to /content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/processual_class_trainer_output/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/processual_class_trainer_output/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/processual_class_trainer_output/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/processual_class_trainer_output/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to /content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/processual_class_trainer_output/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its sta

TrainOutput(global_step=11598, training_loss=0.10807358577223065, metrics={'train_runtime': 22324.5257, 'train_samples_per_second': 1.039, 'train_steps_per_second': 0.52, 'total_flos': 0.0, 'train_loss': 0.10807358577223065, 'epoch': 3.0})

In [ ]:
validation_metrics = trainer.evaluate()
validation_metrics

***** Running Evaluation *****
  Num examples = 1658
  Batch size = 2


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 3.0,
 'eval_loss': 0.1033172607421875,
 'eval_runtime': 420.3724,
 'eval_samples_per_second': 3.944,
 'eval_steps_per_second': 1.972,
 'eval_validation_accuracy': 0.7599517490952955,
 'eval_validation_f1': 0.7350943941925584,
 'eval_validation_precision': 0.7688367014146309,
 'eval_validation_recall': 0.7599517490952955}

In [ ]:
torch.save(model.state_dict(), '/content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/saved_models/processual_class_frontback_model.bin')

## Avaliação no Conjunto de Testes

Terminada a fase de treinamento do modelo, vamos avaliá-lo com os dados do conjunto de testes para poder compará-lo com o modelo FrontBack sem a adição do campo da classe processual.

O procedimento executado é o mesmo do notebook de métricas para comparação das diferentes aproximações de construção dos modelos: o conjunto de testes é carregado do Drive e os dados são tokenizados e depois classificados pelos modelos.

In [ ]:
test_documents = pd.read_csv(CONSTANTS['TEST_DATASET'])
test_documents.dropna(inplace=True, subset=['cod_acordao'])

In [ ]:
# Definition of mapping from law branch name to a numeric identifier

class LawBranch(Enum):
    """Mapping to a Law Branch and an identification. The enum also stores
    the law branch name in a free text form.
    """

    Penal = (0, "Direito Penal (Direito Processual Penal)")
    Administrativo = (1, "Direito Administrativo (Licitações, Contratos Administrativos, Servidores, Desapropriação, Tribunal de Contas, Improbidade, etc.)")
    Tributario = (2, "Direito Tributário/Direito Financeiro")
    Civil = (3, "Direito Civil (Direito Comercial/Direito de Família)")
    Previdenciario = (4, "Direito Previdenciário")
    Trabalho = (5, "Direito do Trabalho")
    Processual_Civil = (6, "Direito Processual Civil")
    Eleitoral = (7, "Direito Eleitoral")
    Consumidor = (8, "Direito do Consumidor")
    Internacional = (9, "Direito Internacional (Público ou Privado)")
    Militar = (10, "Direito Militar")
    Economico = (11, "Direito Econômico (Direito concorrencial e Agências Reguladoras Setoriais, Intervenção no Domínio Econômico)")
    Ambiental = (12, "Direito Ambiental")

    def get_identifier(self) -> int:
        """Retrieves the identifier number for this instance of LawBranch.

        Returns:
            int: identifier of this instance of LawBranch
        """
        return self.value[0]
    
    @staticmethod
    def get_all_names() -> List[str]:
      """Retrieves a list of all names defined in LawBranch enum.

        Returns:
          List[str]: the list of names.

      """
      names = []
      for law_branch in LawBranch:
        names.append(law_branch.name)
      return names

In [ ]:
# Função para tokenizar as ementas
def tokenize(ementas):
  """
    @param    ementas (list): Array de ementas a serem tokenizadas.
    @return   dataloader (torch.utils.data.DataLoader): DataLoader com os dados das predições
  """
  input_ids = []
  attention_masks = []

  for ementa in ementas:
    encoded_dict = tokenizer.encode_plus(
        ementa,
        add_special_tokens = True,
        max_length = HYPERPARAMETERS['MAX_NUMBER_TOKENS'],
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        return_tensors = 'pt',
        return_token_type_ids=False,
    )
        
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)

  prediction_data = TensorDataset(input_ids, attention_masks)
  prediction_sampler = SequentialSampler(prediction_data)
  return DataLoader(prediction_data, sampler=prediction_sampler, batch_size=HYPERPARAMETERS['BATCH_SIZE'])


# Função que chama o modelo para classificar os acórdãos
def classify(acordaos, model):
  """
    @param  acordaos (pd.DataFrame): Dataframe com os acórdãos a serem classificados.
    @param  model (transformers.BertModel): Modelo BERT pré-treinado a ser utilizado.
    @return classifications (dict): Dicionário com os acórdãos classificados e suas ementas.
  """
  classifications = {
      'True Label': [],
      'Predicted Label': [],
      'Ementa': []
  }

  for id, row in acordaos.iterrows():
    classifications['Ementa'].append(row['ementa'])
    classifications['True Label'].append(row['ramo'])
  prediction_dataloader = tokenize(classifications['Ementa'])
  
  model.eval()
  predictions = []
  for batch in prediction_dataloader:
    batch = tuple(t.to(device) for t in batch)
    
    b_input_ids, b_input_mask = batch

    with torch.no_grad():
        _, outputs_logits = model(b_input_ids, attention_mask=b_input_mask)

    logits = outputs_logits
    logits = logits.detach().cpu().numpy()    
    predictions.append(logits)

  for prediction_batch in predictions:
    predicted_labels = np.argmax(prediction_batch, axis=1).flatten()
    for prediction in predicted_labels:
        classifications['Predicted Label'].append(prediction)

  return classifications

In [ ]:
frontback_model = LawDocumentClassifier(HYPERPARAMETERS['NUMBER_OF_BRANCHES'], len(documents_train) // HYPERPARAMETERS['BATCH_SIZE'], HYPERPARAMETERS['EPOCHS'], HYPERPARAMETERS['LEARNING_RATE'])
frontback_model.load_state_dict(torch.load(CONSTANTS['MODEL_PATH']))

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from th

<All keys matched successfully>

In [ ]:
model = LawDocumentClassifier(HYPERPARAMETERS['NUMBER_OF_BRANCHES'], len(documents_train) // HYPERPARAMETERS['BATCH_SIZE'], HYPERPARAMETERS['EPOCHS'], HYPERPARAMETERS['LEARNING_RATE'])
model.load_state_dict(torch.load('/content/drive/My Drive/MAC499 - Kaique e Yurick/Projeto/saved_models/processual_class_frontback_model.bin'))

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from th

<All keys matched successfully>

In [ ]:
model.cuda()
proc_class_classifications = classify(test_documents, model)
frontback_model.cuda()
frontback_classifications = classify(test_documents, frontback_model)

In [ ]:
print(classification_report(proc_class_classifications['True Label'], proc_class_classifications['Predicted Label'], target_names=LawBranch.get_all_names()))

                  precision    recall  f1-score   support

           Penal       0.96      0.94      0.95       334
  Administrativo       0.81      0.86      0.84       259
      Tributario       0.73      0.76      0.74        46
           Civil       0.00      0.00      0.00        12
  Previdenciario       0.62      0.56      0.59        32
        Trabalho       0.44      0.53      0.48        15
Processual_Civil       0.50      0.52      0.51        48
       Eleitoral       0.67      0.40      0.50         5
      Consumidor       1.00      0.25      0.40         4
   Internacional       0.88      0.96      0.92        53
         Militar       0.60      0.69      0.64        13
       Economico       0.00      0.00      0.00         3
       Ambiental       0.75      0.60      0.67         5

        accuracy                           0.83       829
       macro avg       0.61      0.55      0.56       829
    weighted avg       0.82      0.83      0.83       829



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(classification_report(frontback_classifications['True Label'], frontback_classifications['Predicted Label'], target_names=LawBranch.get_all_names()))

                  precision    recall  f1-score   support

           Penal       0.98      0.94      0.96       334
  Administrativo       0.82      0.88      0.85       259
      Tributario       0.69      0.76      0.72        46
           Civil       0.40      0.17      0.24        12
  Previdenciario       0.69      0.62      0.66        32
        Trabalho       0.55      0.40      0.46        15
Processual_Civil       0.48      0.48      0.48        48
       Eleitoral       0.75      0.60      0.67         5
      Consumidor       1.00      0.25      0.40         4
   Internacional       0.85      0.96      0.90        53
         Militar       0.65      0.85      0.73        13
       Economico       0.00      0.00      0.00         3
       Ambiental       1.00      0.60      0.75         5

        accuracy                           0.84       829
       macro avg       0.68      0.58      0.60       829
    weighted avg       0.84      0.84      0.83       829



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
proc_class_metrics = {
    "Balanced Accuracy": balanced_accuracy_score(proc_class_classifications['True Label'], proc_class_classifications['Predicted Label']),
    "Cohen Kappa Score": cohen_kappa_score(proc_class_classifications['True Label'], proc_class_classifications['Predicted Label']),
    "Matthews Correlation Coefficient": matthews_corrcoef(proc_class_classifications['True Label'], proc_class_classifications['Predicted Label'])
}
print(proc_class_metrics)

{'Cohen Kappa Score': 0.7684641366658562, 'Matthews Correlation Coefficient': 0.7688757040628372}


In [ ]:
frontback_metrics = {
    "Balanced Accuracy": balanced_accuracy_score(frontback_classifications['True Label'], frontback_classifications['Predicted Label']),
    "Cohen Kappa Score": cohen_kappa_score(frontback_classifications['True Label'], frontback_classifications['Predicted Label']),
    "Matthews Correlation Coefficient": matthews_corrcoef(frontback_classifications['True Label'], frontback_classifications['Predicted Label'])
}
print(frontback_metrics)

{'Cohen Kappa Score': 0.7787993933168555, 'Matthews Correlation Coefficient': 0.7793445797579482}
